In [ ]:
import os
import utils
import connectome_create
# viz_method = one of ['itkwidgets', 'vtk']
viz_method = 'vtk'

# import some of our favorite packages
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import re
import warnings

%matplotlib inline
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.cluster import AgglomerativeClustering
# import nglui.statebuilder as ngstbld

# # this is the EM specific package for querying the EM data
from caveclient import CAVEclient


# from meshparty import trimesh_io, trimesh_vtk
# from meshparty import skeletonize, skeleton_io, skeleton
import cloudvolume

%load_ext autoreload
%autoreload 2

# Load Premotor table

In [ ]:
pre_to_df = connectome_create.load_pre_to_df(ext='ordered_no_frags')
pre_to_df.loc['motor'].index.to_frame().set_index('preferred_pool').to_csv('./dfs_saved/mn_to_pool_match.csv')

In [ ]:
pre_to_mn_df = connectome_create.load_pre_to_mn_df(ext='matched_typed_with_nt')

In [ ]:
pool_keys = [
        'thorax_swing',
        'thorax_stance',
        'trochanter_extension',
        'trochanter_flexion',
        'femur_reductor',
        'tibia_extensor',
        'main_tibia_flexor',
        # 'auxiliary_tibia_flexor_A',
        'auxiliary_tibia_flexor_B',
        'auxiliary_tibia_flexor_E',
        'ltm',
        'tarsus_depressor_med_venU',
        'tarsus_depressor_noid',
        ]

# input: ['cell_class', 'preferred_pool', 'NT', 'classification_system', 'cell_type', 'segID']
# output: ['cell_class', 'preferred_pool', 'NT', 'classification_system', 'cell_type', 'post_pt_root_id']

# Plot sensory neurons

In [ ]:
All = slice(None)
input_tup = ('sensory',All,All,All,All,All)
output_tup = ('motor',All,All,All,All,All)

pre_df_slice = pre_to_df.loc[output_tup,input_tup]

%config InlineBackend.figure_formats = ['png']
%matplotlib inline

cmap = utils.white_dense()

fig = plt.figure(1, figsize = [8,16])
ax = sns.heatmap(np.log10(pre_df_slice.T.to_numpy()+1), cmap=cmap)
cbar = ax.collections[0].colorbar
# here set the labelsize by 20
# cbar.ax.tick_params(labelsize=20)
cbar.set_label(label = 'log 10 # of synapses', size=24)
plt.xlabel('Motor Neurons', fontsize =18)
plt.ylabel('All to motor', fontsize =18)
plt.yticks(fontsize = 16)
plt.xticks(fontsize = 16)
plt.show()

# Not enough here. 

In [ ]:
%config InlineBackend.figure_formats = ['png']
%matplotlib inline

cmap = utils.white_dense()
def conn_heat_map(df,xticks,yticks,figsz=[16,16],savefig=False,figfilename=None,cmap=cmap):
    fig = plt.figure(1, figsize = figsz)
    ax = sns.heatmap(np.log10(df.T.to_numpy()+1), xticklabels=xticks, yticklabels=yticks, cmap=cmap)
    cbar = ax.collections[0].colorbar
    # here set the labelsize by 20
    # cbar.ax.tick_params(labelsize=20)
    cbar.set_label(label = 'log 10 # of synapses', size=24)
    plt.xlabel('Onto Output neurons', fontsize =18)
    plt.ylabel('From input neurons', fontsize =18)
    plt.yticks(fontsize = 16)
    plt.xticks(fontsize = 16)
    plt.show()
    if savefig:
        if figfilename is None:
            figfilename = 'flexor_pool_temp_' + (np.random.randint(10000,99999,size=None)).astype(str)
        fig.savefig('./figpanels/' + figfilename + '.svg',format='svg')

def lin_heat_map(df,xticks,yticks,figsz=[16,16],savefig=False,figfilename=None,cmap=cmap):
    fig = plt.figure(1, figsize = figsz)
    ax = sns.heatmap(df.T.to_numpy(), xticklabels=xticks, yticklabels=yticks, cmap=cmap)
    cbar = ax.collections[0].colorbar
    # here set the labelsize by 20
    # cbar.ax.tick_params(labelsize=20)
    cbar.set_label(label = 'linear', size=24)
    plt.xlabel('Onto Output neurons', fontsize =18)
    plt.ylabel('From input neurons', fontsize =18)
    plt.yticks(fontsize = 16)
    plt.xticks(fontsize = 16)
    plt.show()
    if savefig:
        if figfilename is None:
            figfilename = 'flexor_pool_temp_' + (np.random.randint(10000,99999,size=None)).astype(str)
        fig.savefig('./figpanels/' + figfilename + '.svg',format='svg')

# Use the motory-ness routine to look at sensory neurons

In [ ]:
def get_cum_motor_premotor(pre_df,c_tup):
    ct_df = pre_df.loc[:,c_tup] # a more generalized version of the above

    # pd.Series

    # pd.DataFrame of fractional inputs
    cell_classes = ct_df.index.get_level_values('cell_class').unique()
    frac_df = pd.DataFrame(index=cell_classes,columns=ct_df.columns)
    for key in cell_classes:
        frac_df.loc[key,:] = ct_df.loc[key,:].sum(axis=0)        

    ct_to_all    = ct_df.loc[All,:].sum(axis=0)                         # calculate the total
    x = [i for i in range(len(ct_df.loc['motor',:].sum(axis=0)))]   
    frac_df.loc['x',:] = x                                              # this x is the original order of the cell_class
    frac_df.loc['total',:] = ct_to_all

    otherpre = ['descending', 
                'sensory', 
                'ascending', 
                'intersegmental',]
                
    frac_df.loc['other_pre',:] = frac_df.loc[otherpre,:].sum(axis=0)


    otherlist = ['non_t1_motor', 
                'descending_post', 
                'sensory_post',
                'ascending_post', 
                'neurmodulatory_ascending_neuron',
                'vnc_non_premotor']
    
    frac_df.loc['other',:] = frac_df.loc[otherlist,:].sum(axis=0)
    frac_df.loc['other',:] = frac_df.loc['other',:]/ct_to_all 
    frac_df = frac_df.drop(otherlist,axis=0)
    
    frac_df = frac_df.sort_values(['other'],axis=1,ascending=True)    # sort according to how other fraction
    sorted_total=frac_df.loc['total',:].to_numpy()                      # keep the sorted total 
    frac_df = frac_df.drop('total',axis=0)                             # drop the total row

    frac_df.loc['cum_motor',:] = frac_df.loc['motor',:].cumsum()
    frac_df.loc['cum_motor_frac',:] = frac_df.loc['cum_motor',:]/frac_df.loc['motor',:].sum()

    frac_df.loc['cum_local',:]          = frac_df.loc['local',:].cumsum()
    frac_df.loc['cum_descending',:]     = frac_df.loc['descending',:].cumsum()
    frac_df.loc['cum_sensory',:]        = frac_df.loc['sensory',:].cumsum()
    frac_df.loc['cum_intersegmental',:] = frac_df.loc['intersegmental',:].cumsum()
    frac_df.loc['cum_other_pre',:] = frac_df.loc['other_pre',:].cumsum()
    

    # clean up a little
    frac_df.loc['cum_motor',frac_df.loc['cum_motor',:]==0] = np.nan
    frac_df.loc['cum_local',frac_df.loc['cum_local',:]==0] = np.nan
    frac_df.loc['cum_other_pre',frac_df.loc['cum_other_pre',:]==0] = np.nan

    # frac_df.loc['cum_motor_over_pre',:] = frac_df.loc['cum_motor',:]/frac_df.loc['cum_local',:]
    frac_df.loc['cum_local_over_motor',:] = frac_df.loc['cum_local',:]/frac_df.loc['cum_motor',:]
    frac_df.loc['cum_inter_over_motor',:] = frac_df.loc['cum_intersegmental',:]/frac_df.loc['cum_motor',:]
    frac_df.loc['cum_other_over_motor',:] = frac_df.loc['cum_other_pre',:]/frac_df.loc['cum_motor',:]

    # frac_df.loc['cum_local_over_motor',frac_df.loc['cum_local_over_motor',:]>10] = np.nan
    # frac_df.loc['cum_other_over_motor',frac_df.loc['cum_other_over_motor',:]>10] = np.nan

    return frac_df, sorted_total



# Look at sensory input on to local neurons compared to mns

In [ ]:
sensory = pre_to_df.loc[:,('sensory',All,All,All,All)]
frac_df, sorted_total = get_cum_motor_premotor(pre_to_df,('sensory',All,All,All,All))
sens_motor_df = frac_df.sort_values(['motor'],axis=1,ascending=False)

# find the indices of the top 5 sensory neurons in frac_df
idx = []
sens_idx = idx
for rtid in sens_motor_df.columns.get_level_values('segID').to_list()[0:9]:
    print(rtid)
    idx = idx+[frac_df.columns.get_locs((All,All,All,All,All,rtid))[0]]
    sens_idx = sens_idx+[sensory.columns.get_locs((All,All,All,All,All,rtid))[0]]

print(idx)
print(sens_idx)

In [ ]:
motory_cutoff_idx = np.array(idx).max()
motor_cutoff_tup = frac_df.columns[motory_cutoff_idx]

In [ ]:
fig, ax = plt.subplots(2, 2, figsize=(10,8),gridspec_kw={'width_ratios': [2, 1,]})
xdim = pre_to_df.loc[:,('sensory',All,All,All,All)].shape[1]
x = [i for i in range(len(sorted_total))]
ax[0,0].plot(x,frac_df.loc['cum_local_over_motor',:],color="#7D688E",label='local')
ax[0,0].plot(x,frac_df.loc['cum_inter_over_motor',:],color="#7D688E",label='local')
ax[0,0].set_xlim(0,xdim)
ax[0,0].set_ylim(0,6)
ax[1,0].plot(x,frac_df.loc['cum_motor',:],color='#910951')
ax[1,0].set_xlim(0,xdim)
ax[1,0].set_ylim(0,frac_df.loc['cum_motor',:].max()*1.05)

ax[0,1].plot(frac_df.loc['motor'],frac_df.loc['local']/frac_df.loc['motor'],color='#cccccc',marker='.',ls='none')
ax[0,0].plot(np.array([motory_cutoff_idx,motory_cutoff_idx]),np.array([0,6]),color='#cccccc')

ax[1,1].plot(frac_df.loc['motor'],frac_df.loc['local']/frac_df.loc['motor'],color='#cccccc',marker='.',ls='none')

for i in idx:
    ax[1,0].plot(x[i],frac_df.loc['cum_motor'].iloc[i],color='#910951',marker='o')
    ax[0,1].plot(frac_df.loc['motor'].iloc[i],frac_df.loc['local'].iloc[i]/frac_df.loc['motor'].iloc[i],color='#910951',marker='o')
    ax[1,1].plot(frac_df.loc['motor'].iloc[i],frac_df.loc['local'].iloc[i]/frac_df.loc['motor'].iloc[i],color='#910951',marker='o')

# ax[1,1].set_xlim(0,)
ax[1,1].set_ylim(0,6)

# Input to common local circuitry
Consider those desc neurons that tend to target T1 stuff, ie. less onto other stuff than the 5 outliers. What is the ratio of preferred pool input to other pools?

In [ ]:
def keep_first_string_in_list(in_list):
    out_list = []
    key = None
    for s in in_list:
        if key is None:
            key = s
            out_list = out_list + [key]
            continue
        elif s==key:
            out_list = out_list + ['']
        else:
            key = s
            out_list = out_list + [s]
    return out_list

In [ ]:
motor_pool_df = sensory.loc['motor']
motor_pool_df = motor_pool_df.groupby(by='preferred_pool',axis=0).sum()
motor_pool_df = motor_pool_df.loc[pool_keys,:]

motor_targets_df = sensory.loc['motor']
motor_targets_df = motor_targets_df.groupby(by='preferred_pool',axis=0).sum()
motor_targets_df = motor_targets_df/motor_targets_df.sum(axis=0)
motor_targets_df = motor_targets_df.loc[pool_keys,:]
motor_targets_df.loc['pref_pool_input'] = np.nan
for tup in motor_targets_df.columns.to_list():
    motor_targets_df.loc['pref_pool_input',tup] = motor_targets_df.loc[tup[1],tup]

local_targets_df = sensory.loc['local']
local_targets_df = local_targets_df.groupby(by='preferred_pool',axis=0).sum()
local_targets_df = local_targets_df/local_targets_df.sum(axis=0)
local_targets_df = local_targets_df.loc[pool_keys,:]

local_targets_df.loc['pref_pool_input'] = np.nan
for tup in local_targets_df.columns.to_list():
    local_targets_df.loc['pref_pool_input',tup] = local_targets_df.loc[tup[1],tup]


# yticklabels = [x + '_' + str(y) for x,y in zip(local_targets_df.columns.get_level_values('preferred_pool'),local_targets_df.columns.get_level_values('segID'))]
yticklabels = keep_first_string_in_list(local_targets_df.columns.get_level_values('preferred_pool').to_list())

conn_heat_map(df=motor_pool_df, xticks=local_targets_df.index.get_level_values('preferred_pool'),yticks=yticklabels,figsz=[4,16])
lin_heat_map(df=motor_targets_df, xticks=local_targets_df.index.get_level_values('preferred_pool'),yticks=yticklabels,figsz=[4,16])
lin_heat_map(df=local_targets_df, xticks=local_targets_df.index.get_level_values('preferred_pool'),yticks=yticklabels,figsz=[4,16])

In [ ]:
sensory.max().max()

## Pool motoryness vs. pool pre-motory-ness?

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(10,8)) #,gridspec_kw={'width_ratios': [2, 1,],'height_ratios': [2,.2]})
ax.plot(motor_targets_df.loc['pref_pool_input'],local_targets_df.loc['pref_pool_input'],color="#cccccc",label='local',marker='.',ls='none')

for i in sens_idx:
    ax.plot(motor_targets_df.loc['pref_pool_input'].iloc[i],local_targets_df.loc['pref_pool_input'].iloc[i],color="#910951",label='local',marker='o',ls='none')

ax.set_xlabel('Pool preference strength - MNs')
ax.set_ylabel('Pool preference strength - local')
print(sensory.loc['motor',((motor_targets_df.loc['pref_pool_input']==1) & (local_targets_df.loc['pref_pool_input']==1))].sum(axis=0))

pd.concat([motor_targets_df.loc[['pref_pool_input']],local_targets_df.loc[['pref_pool_input']]],axis=0).T.corr()

## Sensory neurons vs. HL?

In [ ]:
## Create lookuptable
cell_types = pre_to_mn_df.index.get_level_values('cell_type').unique().to_numpy()
nt_table = pd.read_csv('./annotations_hl/LacinHLTable.csv')
import re
pattern = re.compile("R[0-9]")

hl_lut = {'cell_type':cell_types,'hemilineage':cell_types}
hl_lut = pd.DataFrame(data=hl_lut)

for idx,r in hl_lut.iterrows():
    try:
        if np.isnan(r.hemilineage):
            continue
    except TypeError:
        pass
    hl = r.hemilineage

    if hl[0:2]=='RD':
        hl = '24B'
        # print('string was RD, now {}'.format(hl))

    if pattern.match(hl) or hl=='RVD' or hl=='Rcore_':
        # print('string was {}, now 8A'.format(hl))
        hl = '8A'

    if not hl.isalnum():
        hl = hl[0:-1]
        # print(hl)
    
    if not hl.isalnum():
        hl=hl[0:hl.find('_')]
        # print(hl)

    if hl == '9Ac':
        hl='9A'
        # print(hl)

    hl_lut.loc[idx,'hemilineage'] = hl

# merge with neurotransmitter table to get a lookuptable
hl_lut = hl_lut.merge(nt_table,how='outer',left_on='hemilineage',right_on='HL')
hl_lut = hl_lut.loc[~hl_lut.hemilineage.isna(),:]

hl_lut_reduced = hl_lut[['cell_type','NT','hemilineage']]


In [ ]:
# simplify the hemilineages
local_targets_df = sensory.loc['local']

# just operate on the index, where the nt will be
pmn_index_df = local_targets_df.index.to_frame().reset_index(drop=True)

# join the index on cell_type, with NT
pmn_index_df = pmn_index_df.join(hl_lut_reduced.set_index('cell_type'),how='left',on='cell_type',lsuffix='_x',rsuffix='_y')

# reorder the matrix
pmn_index_df = pmn_index_df[['preferred_pool','NT_x','classification_system','cell_type','hemilineage','post_pt_root_id']]
pmn_index_df.hemilineage.value_counts()

local_targets_df.index = pd.MultiIndex.from_frame(pmn_index_df)
local_targets_df = local_targets_df.groupby(by='hemilineage',axis=0).sum()

hl_preference_df = local_targets_df.copy()
# local_targets_df.loc['pref_HL'] = np.nan

for tup in local_targets_df.columns.to_list():
    # print(tup)
    hl_preference_df.loc[:,tup] = local_targets_df.loc[:,tup]/local_targets_df.loc[:,tup].sum(axis=0)
    # local_targets_df.loc['pref_HL',tup] = local_targets_df.loc[:,tup].max(axis=0)/local_targets_df.loc[:,tup].sum(axis=0)

yticklabels = keep_first_string_in_list(local_targets_df.columns.get_level_values('preferred_pool').to_list())

conn_heat_map(df=local_targets_df, xticks=local_targets_df.index.get_level_values('hemilineage'),yticks=yticklabels,figsz=[10,16])
lin_heat_map(df=hl_preference_df, xticks=local_targets_df.index.get_level_values('hemilineage'),yticks=yticklabels,figsz=[10,16])

## Sensory vs NT?

In [ ]:
# simplify the hemilineages
local_targets_df = sensory.loc['local']
local_targets_df = local_targets_df.groupby(by='NT',axis=0).sum()

nt_preference_df = local_targets_df.copy()
# local_targets_df.loc['pref_HL'] = np.nan

for tup in local_targets_df.columns.to_list():
    # print(tup)
    nt_preference_df.loc[:,tup] = local_targets_df.loc[:,tup]/local_targets_df.loc[:,tup].sum(axis=0)
    # local_targets_df.loc['pref_HL',tup] = local_targets_df.loc[:,tup].max(axis=0)/local_targets_df.loc[:,tup].sum(axis=0)

yticklabels = keep_first_string_in_list(local_targets_df.columns.get_level_values('preferred_pool').to_list())

conn_heat_map(df=local_targets_df, xticks=local_targets_df.index.get_level_values('NT'),yticks=yticklabels,figsz=[1,16])
lin_heat_map(df=nt_preference_df, xticks=local_targets_df.index.get_level_values('NT'),yticks=yticklabels,figsz=[1,16])

In [ ]:
nt_preference_df

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(10,8)) #,gridspec_kw={'width_ratios': [2, 1,],'height_ratios': [2,.2]})
ax.plot(local_targets_df.loc['Ach'],local_targets_df.loc['Glu'],color="#000000",label='local',marker='.',ls='none')
ax.plot([0, local_targets_df.loc['Ach'].max()],[0, local_targets_df.loc['Ach'].max()],color='#cccccc' )
ax.set_aspect('equal', 'box')
ax.set_xlabel('Ach connections')
ax.set_ylabel('Glu connections')


In [ ]:

for i in desc_idx:
    ax.plot(motor_targets_df.loc['pref_pool_input'].iloc[i],local_targets_df.loc['pref_pool_input'].iloc[i],color="#910951",label='local',marker='o',ls='none')

ax.set_xlabel('Pool preference strength - MNs')
ax.set_ylabel('Pool preference strength - local')
print(descending.loc['motor',((motor_targets_df.loc['pref_pool_input']==1) & (local_targets_df.loc['pref_pool_input']==1))].sum(axis=0))

pd.concat([motor_targets_df.loc[['pref_pool_input']],local_targets_df.loc[['pref_pool_input']]],axis=0).T.corr()

# Take a look at the biggest descending neurons

In [ ]:
All = slice(None)
input_tup = ('sensory',All,All,All,All,sens_motor_df.columns.get_level_values('segID').to_list()[0:9])
output_tup = ('motor',All,All,All,All,All)

big_desc_df = pre_to_df.loc[output_tup,input_tup]
mnpoollabel = big_desc_df.index.get_level_values('preferred_pool')
yticklabels = [x + '_' + str(y) for x,y in zip(big_desc_df.columns.get_level_values('preferred_pool'),big_desc_df.columns.get_level_values('segID'))]
conn_heat_map(df=big_desc_df, xticks=mnpoollabel,yticks=yticklabels,figsz=[16,4])


# Other

In [ ]:
# How many intersegmental neurons? # ~350
# segID	motor	has_soma	sensory	 neck	local	

def get_motory_fraction(pre_df,c_tup,compress_other_classes=False):
    ct_df = pre_df.loc[:,c_tup] # a more generalized version of the above

    # pd.Series

    # pd.DataFrame of fractional inputs
    cell_classes = ct_df.index.get_level_values('cell_class').unique()
    frac_df = pd.DataFrame(index=cell_classes,columns=ct_df.columns)

    for key in cell_classes:
        frac_df.loc[key,:] = ct_df.loc[key,:].sum(axis=0)

    
    if compress_other_classes:
        otherlist = ['non_t1_motor', 
                    'descending_post', 
                    'sensory_post',
                    'ascending_post', 
                    'neurmodulatory_ascending_neuron',
                    'vnc_non_premotor']
        frac_df.loc['other',:] = frac_df.loc[otherlist,:].sum(axis=0)
        
    ct_to_all    = ct_df.loc[All,:].sum(axis=0)                         # calculate the total
    frac_df = frac_df/ct_to_all                                         # divide by the total
    x = [i for i in range(len(ct_df.loc['motor',:].sum(axis=0)))]   
    frac_df.loc['x',:] = x                                              # this x is the original order of the cell_class
    frac_df.loc['total',:] = ct_to_all
    frac_df = frac_df.sort_values(['motor'],axis=1,ascending=False)    # sort according to how t1 motory

    sorted_total=frac_df.loc['total',:].to_numpy()                      # keep the sorted total 
    frac_df  = frac_df.drop('total',axis=0)                             # drop the total row
    
    if compress_other_classes:
        frac_df = frac_df.drop(otherlist,axis=0)
        frac_df = frac_df.sort_values(['other'],axis=1,ascending=False)    # sort according to how t1 motory
    
    return frac_df, sorted_total                                       


## First get the motoryness of the Tibia extensor, flexor pool

In [ ]:
# first sort by motoryness
input_tup = ('local',['tibia_extensor','main_tibia_flexor'],All,All,All,All,All)
frac_df,sorted_total = get_motory_fraction(pre_to_df,input_tup,compress_other_classes=False)

pre_onto_ti_ext_flex_df = pre_to_df.loc[:,input_tup]
pre_onto_ti_ext_flex_df = pre_onto_ti_ext_flex_df.loc[:,frac_df.columns]
pre_onto_ti_ext_flex_df

# # locs, labels = plt.xticks(ticks=x, labels=lbls, rotation=90)
          #'motor', 'nont1',    'desc',     'sens',     'ascend',   'local',    'intr',     'vnc',      'fragments'
colors = ["#910951","#CC4893",  "#79D0F7",  "#93E5B6",  "#6084CC",  "#7D688E",  "#CC9FCC",  '#5C085E',   '#cccccc']
cmap = sns.set_palette(sns.color_palette(colors))
x = [i for i in range(len(sorted_total))]
ax = frac_df.T.plot(x='x',kind='bar', stacked=True, legend = True,width=1,cmap=cmap,figsize=(16,40)) 

## Show the Motor neurons first

In [ ]:
# input_tup = ('local',['tibia_extensor','main_tibia_flexor'],All,All,All,All,All)
output_tup = ('motor',['tibia_extensor','main_tibia_flexor'],All,All,All,All,All)

pre_onto_ti_ext_flex_df = pre_onto_ti_ext_flex_df.loc[output_tup,:]
# weird neurons
print('no NT: {}'.format(pre_onto_ti_ext_flex_df.loc[:,~pre_onto_ti_ext_flex_df.columns.get_level_values('NT').isin(['Ach','GABA','Glu'])].columns.get_level_values('segID').to_list()))
# Get rid of the last two pMNs
pre_onto_ti_ext_flex_df = pre_onto_ti_ext_flex_df.loc[:,pre_onto_ti_ext_flex_df.columns.get_level_values('NT').isin(['Ach','GABA','Glu'])]

# xticklabels = pre_onto_ti_ext_flex_df.index.get_level_values('preferred_pool')
# yticklabels = pre_df_ti_ext_flex.columns.get_level_values('NT')

# conn_heat_map(df=pre_df_ti_ext_flex, xticks=xticklabels,yticks=yticklabels,figsz=[4,16])

# order by NT
cols_ntsorted,c_array = pre_onto_ti_ext_flex_df.columns.sortlevel(level='NT',ascending=True,sort_remaining=False)
mn_nt_sorted = pre_onto_ti_ext_flex_df.loc[:,cols_ntsorted]

# order by pool
cols_ntsorted,c_array = mn_nt_sorted.columns.sortlevel(level='preferred_pool',ascending=True,sort_remaining=False)
output_tup = ('local',['tibia_extensor','main_tibia_flexor'],All,All,All,All,All)
mn_nt_pool_sorted = mn_nt_sorted.loc[:,output_tup]

mnpoollabel = mn_nt_pool_sorted.index.get_level_values('preferred_pool')
yticklabels = mn_nt_pool_sorted.columns.get_level_values('NT')

conn_heat_map(df=mn_nt_pool_sorted, xticks=mnpoollabel,yticks=yticklabels,figsz=[4,16])

## Order local neurons by NT, and pool

In [ ]:
# input_tup = ('local',['tibia_extensor','main_tibia_flexor'],All,All,All,All)
output_tup = ('local',['tibia_extensor','main_tibia_flexor'],All,All,All,All,All)

pre_onto_ti_ext_flex_df = pre_onto_ti_ext_flex_df.loc[output_tup,:]
# weird neurons
print('no NT: {}'.format(pre_onto_ti_ext_flex_df.loc[:,~pre_onto_ti_ext_flex_df.columns.get_level_values('NT').isin(['Ach','GABA','Glu'])].columns.get_level_values('segID').to_list()))
# Get rid of the last two pMNs
pre_onto_ti_ext_flex_df = pre_onto_ti_ext_flex_df.loc[pre_onto_ti_ext_flex_df.index.get_level_values('NT').isin(['Ach','GABA','Glu']),pre_onto_ti_ext_flex_df.columns.get_level_values('NT').isin(['Ach','GABA','Glu'])]

# sort by 
rows_ntsorted,array = pre_onto_ti_ext_flex_df.index.sortlevel(level='NT',ascending=True,sort_remaining=False)
cols_ntsorted,c_array = pre_onto_ti_ext_flex_df.columns.sortlevel(level='NT',ascending=True,sort_remaining=False)
local_nt_sorted = pre_onto_ti_ext_flex_df.loc[rows_ntsorted,cols_ntsorted]

# order by pool
output_tup = ('local',['tibia_extensor','main_tibia_flexor'],All,All,All,All,All)
local_nt_pool_sorted = local_nt_sorted.loc[output_tup,output_tup]

# mnpoollabel = local_nt_pool_sorted.index.get_level_values('preferred_pool')
yticklabels = local_nt_pool_sorted.columns.get_level_values('NT')

xpool = local_nt_pool_sorted.index.get_level_values('preferred_pool')
pool_dict = {'tibia_extensor':'ext','main_tibia_flexor':'flex'}
xpool = [pool_dict[key] for key in xpool]

xNT = local_nt_pool_sorted.index.get_level_values('NT').to_list()
xpoolnt = [x + "_" + y for x,y in zip(xpool,xNT)]

conn_heat_map(df=local_nt_pool_sorted, xticks=xpoolnt,yticks=xpoolnt,figsz=[16,16])

In [ ]:
# Format a nicer figure with both plots side by side
# do it in different colors too, then cut out different versions in illustrator
fig, ax = plt.subplots(1, 2, figsize=(32,16),gridspec_kw={'width_ratios': [1, 3.5]})

cmap = utils.white_dense()
# cmap = sns.color_palette("light:seagreen", as_cmap=True)

# def conn_heat_map(df,xticks,yticks,figsz=[16,16],savefig=False,figfilename=None,cmap=cmap):
sns.heatmap(np.log10(mn_nt_pool_sorted.T.to_numpy()+1),ax=ax[0], xticklabels=mnpoollabel, yticklabels=xpoolnt, cmap=cmap,square=True,cbar=False,vmax=np.log10(633),vmin=0)
plt.xlabel('Onto MNs', fontsize =18)
plt.ylabel('From input neurons', fontsize =18)
sns.heatmap(np.log10(local_nt_pool_sorted.T.to_numpy()+1),ax=ax[1], xticklabels=xpoolnt, yticklabels=xpoolnt,square=True, cmap=cmap,vmax=np.log10(633),vmin=0)
plt.xlabel('Onto local pmns', fontsize =18)

cbar = ax[1].collections[0].colorbar
cbar.set_label(label = 'log 10 # of synapses', size=24)
# plt.yticks(fontsize = 16)
# plt.xticks(fontsize = 16)
# plt.show()

fig.savefig('./figpanels/' + 'local_ti_ext_flex_by_nt' + '.svg',format='svg')


In [ ]:
mn_nt_pool_sorted.max().max()